In [1]:
import pandas as pd
import os
import unittest
import string
import re
pd.set_option('display.max_rows', 10)

In [2]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day7_input.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s = file1.read().splitlines()
pd.Series(s)

0      dark olive bags contain 2 muted brown bags, 1 ...
1      faded coral bags contain 3 drab cyan bags, 1 l...
2          plaid plum bags contain 2 mirrored cyan bags.
3      clear maroon bags contain 1 dotted turquoise b...
4      plaid coral bags contain 3 posh fuchsia bags, ...
                             ...                        
589          bright plum bags contain 5 wavy brown bags.
590         pale coral bags contain 3 bright olive bags.
591    dim fuchsia bags contain 2 bright chartreuse b...
592    dim cyan bags contain 2 faded tan bags, 4 mirr...
593    faded cyan bags contain 1 dotted black bag, 1 ...
Length: 594, dtype: object

In [3]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day7_example1.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s_ex1 = file1.read().splitlines()
pd.Series(s_ex1)

0    light red bags contain 1 bright white bag, 2 m...
1    dark orange bags contain 3 bright white bags, ...
2          bright white bags contain 1 shiny gold bag.
3    muted yellow bags contain 2 shiny gold bags, 9...
4    shiny gold bags contain 1 dark olive bag, 2 vi...
5    dark olive bags contain 3 faded blue bags, 4 d...
6    vibrant plum bags contain 5 faded blue bags, 6...
7               faded blue bags contain no other bags.
8             dotted black bags contain no other bags.
dtype: object

In [4]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day7_example2.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file2 = open(trainFile, 'r') 
s_ex2 = file2.read().splitlines()
pd.Series(s_ex1)

0    light red bags contain 1 bright white bag, 2 m...
1    dark orange bags contain 3 bright white bags, ...
2          bright white bags contain 1 shiny gold bag.
3    muted yellow bags contain 2 shiny gold bags, 9...
4    shiny gold bags contain 1 dark olive bag, 2 vi...
5    dark olive bags contain 3 faded blue bags, 4 d...
6    vibrant plum bags contain 5 faded blue bags, 6...
7               faded blue bags contain no other bags.
8             dotted black bags contain no other bags.
dtype: object

In [5]:
def splitPart(part):
    subparts = part.split(" ")
    count = int(subparts[0])
    rest = "".join(subparts[1:])
    return (rest, count)

def parseRule(line):
    line = line.replace("bag.","bags")
    line = line.replace(".","")
    line = line.replace("bag,","bags,")    
    line = line.replace(" contain",",")
    line = line.replace(", no other bags","")
    line = "1 " + line
    parts = line.split(", ")
    key = splitPart(parts[0])
    values = dict(map(splitPart, parts[1:]))
    return (key[0],values)

In [6]:
rules_ex1 = dict(map(parseRule, s_ex1))
rules_ex1

{'brightwhitebags': {'shinygoldbags': 1},
 'darkolivebags': {'dottedblackbags': 4, 'fadedbluebags': 3},
 'darkorangebags': {'brightwhitebags': 3, 'mutedyellowbags': 4},
 'dottedblackbags': {},
 'fadedbluebags': {},
 'lightredbags': {'brightwhitebags': 1, 'mutedyellowbags': 2},
 'mutedyellowbags': {'fadedbluebags': 9, 'shinygoldbags': 2},
 'shinygoldbags': {'darkolivebags': 1, 'vibrantplumbags': 2},
 'vibrantplumbags': {'dottedblackbags': 6, 'fadedbluebags': 5}}

In [7]:
rules_ex2 = dict(map(parseRule, s_ex2))
rules_ex2

{'darkbluebags': {'darkvioletbags': 2},
 'darkgreenbags': {'darkbluebags': 2},
 'darkorangebags': {'darkyellowbags': 2},
 'darkredbags': {'darkorangebags': 2},
 'darkvioletbags': {},
 'darkyellowbags': {'darkgreenbags': 2},
 'shinygoldbags': {'darkredbags': 2}}

In [8]:
rules = dict(map(parseRule, s))
pd.Series(rules)

brightaquabags             {'fadedsilverbags': 2, 'dimfuchsiabags': 5}
brightbeigebags      {'mutedbrownbags': 5, 'wavyredbags': 4, 'clear...
brightblackbags      {'lightmaroonbags': 3, 'shinylimebags': 5, 'vi...
brightbluebags                                  {'vibrantplumbags': 4}
brightbronzebags                                {'shinymaroonbags': 3}
                                           ...                        
wavytomatobags                                  {'wavyfuchsiabags': 4}
wavyturquoisebags    {'shinyredbags': 5, 'mirroredyellowbags': 3, '...
wavyvioletbags                                {'dottedfuchsiabags': 5}
wavywhitebags        {'darkbrownbags': 2, 'drabyellowbags': 5, 'dot...
wavyyellowbags              {'dimyellowbags': 2, 'dimlavenderbags': 5}
Length: 594, dtype: object

In [9]:
def containsGoldReference(input, key):
    values = input.get(key)
    for v in values:
        if v == "shinygoldbags":
            return 1
        elif containsGoldReference(input, v):
            return 1
    return 0

def countGoldReferences(input):
    count = 0
    for key in input:
        if containsGoldReference(input,key):
            count += 1
    return count

In [10]:
class PartOneTests(unittest.TestCase):
    def setUp(self):
        pass
    def tearDown(self):
        pass
    def test_examples(self):
        self.assertEqual(4, countGoldReferences(rules_ex1))     

In [11]:
countGoldReferences(rules_ex1)

4

In [12]:
countGoldReferences(rules)

246

In [13]:
def countBags(input, key):
    values = input.get(key)
    count = 0
    for v in values:
        itemCount = values[v]
        count += itemCount * countBags(input, v)
    return count + 1

def countGoldBags(input):
    return countBags(input, "shinygoldbags") - 1

In [14]:
class PartTwoTests(unittest.TestCase):
    def setUp(self):
        pass
    def tearDown(self):
        pass
    def test_examples(self):
        self.assertEqual(32, countGoldBags(rules_ex1))     
        self.assertEqual(126, countGoldBags(rules_ex2))     

In [15]:
unittest.main(argv=[''], verbosity=3, exit=False)

test_examples (__main__.PartOneTests) ... ok
test_examples (__main__.PartTwoTests) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


In [16]:
countGoldBags(rules)

2976